## Import Dependencies

In [9]:
import logging
from abc import ABC, abstractmethod
import math
import json
import importlib
import sys
import configparser
import requests
import os
import openpyxl
import json
from dotenv import load_dotenv
#from pyspark import SparkConf
#from pyspark.sql import SparkSession, DataFrame

## Load Env Variables from .env

In [ ]:
# Load Environment Variables from .env
load_dotenv()
os.environ["SPARK_HOME"] = os.getenv("PROJ_SPARK_HOME","")
project_path = os.getenv("PROJECT_PATH")
dataset_path = f"{project_path}\llm_custom_apps\\datasets"
test_data_path = f"{dataset_path}\\test_datasets"
app_data_path = f"{dataset_path}\\app_datasets"

## Logger Method

In [4]:
def create_logger(logger_nm):
    """
    Function takes a Logger Name and returns a Generic Logger
    :param logger_nm: Name of the Logger that is prefixed to the Log Statements
    :return: logger object to be used across different scripts
    """

    # Define generic logger variable
    gen_logger = logging.getLogger(logger_nm)
    # a) Create Streaming Handler and Set level to Debug
    gen_logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    # b) Create formatter
    lg_str = "\n%(name)s - %(levelname)s - %(lineno)s - %(funcName)s - %(asctime)s - %(message)s"
    formatter = logging.Formatter(lg_str)
    # c) Add formatter to ch
    ch.setFormatter(formatter)
    # d) Add ch to logger after clearing previous handlers
    gen_logger.handlers.clear()
    gen_logger.addHandler(ch)

    return gen_logger

In [5]:
gen_logger = create_logger("Generic Logger")

## Reusable Rest API method

In [6]:
def reusable_rest_api(api_url, req_data=None, cert_verify=False, header=None, method="GET", resp="JSON", retry_lim=3):
    """
    Make HTTP requests to a specified API endpoint with various options.

    Parameters:
    - api_url (str): The URL of the API endpoint to make the request to.
    - req_data (str or dict, optional): The request data to send to the API (e.g., JSON payload). Default is None.
    - cert_verify (bool, optional): Whether to verify the SSL certificate. Default is False.
    - header (dict, optional): Additional headers to include in the request. Default is None.
    - method (str, optional): The HTTP request method to use (GET, POST, PUT, DELETE). Default is "GET".
    - resp (str, optional): The expected response type ("JSON" or "RAW"). Default is "JSON".
    - retry_lim (int, optional): The maximum number of retry attempts in case of connection errors. Default is 3.

    Returns:
    - tuple: A tuple containing the HTTP status code of the response and the response content.

    Raises:
    - requests.exceptions.HTTPError: If an invalid API method is specified.
    - requests.exceptions.ConnectionError: If a connection error occurs, and the maximum retry limit is exceeded.
    - requests.exceptions.Timeout: If a timeout occurs while making the request.
    - requests.exceptions.RequestException: If a general request exception occurs.
    - ValueError: If there is an issue parsing the response content as JSON.

    Example usage:
    status_code, response_data = reusable_rest_api(
        api_url="https://example.com/api",
        req_data={"key": "value"},
        cert_verify=True,
        header={"Authorization": "Bearer Token"},
        method="POST",
        resp="JSON",
        retry_lim=3
    )
    """
    retry = 1
    while True:
        resp_content = ""
        try:
            if method == "GET":
                resp_content = requests.get(api_url, header=header, data=req_data, verify=cert_verify)
                if resp == "JSON": resp_json = json.loads(resp_content.content)
            elif method == "POST":
                resp_content = requests.post(api_url, header=header, data=req_data, verify=cert_verify)
                if resp == "JSON": resp_json = json.loads(resp_content.content)
            elif method == "PUT":
                resp_content = requests.put(api_url, header=header, data=req_data, verify=cert_verify)
                if resp == "JSON": resp_json = json.loads(resp_content.content)
            elif method == "DELETE":
                resp_content = requests.delete(api_url, header=header, data=req_data, verify=cert_verify)
                resp_json = None
            else:
                raise requests.exceptions.HTTPError("Invalid API Method")
        except requests.exceptions.HTTPError as err:
            gen_logger.error(f"The exception while calling this api is: {err}")
            sys.exit(1)
        except requests.exceptions.ConnectionError as err:
            gen_logger.error(f"The exception while calling this api is: {err}")
            retry = retry + 1
            if retry <= retry_lim+1:  continue
            gen_logger.error("Retry Limit Exceeded")
            sys.exit(1)
        except requests.exceptions.Timeout as err:
            gen_logger.error(f"The exception while calling this api is: {err}")
            sys.exit(1)
        except requests.exceptions.RequestException as err:
            gen_logger.error(f"The exception while calling this api is: {err}")
            sys.exit(1)
        except ValueError as err:
            gen_logger.error(f"The exception while calling this api is: {err}")
            retry = retry + 1
            if retry <= retry_lim+1:  continue
            gen_logger.error("Retry Limit Exceeded")
            sys.exit(1)

        final_resp = resp_json if (resp == "JSON") else resp_content
        return resp_content.status_code,final_resp

In [7]:
def sum(a,b):
    return a+b

In [8]:
print(sum(9,4))

13
